In [13]:
# Importation des bibliothèques
import numpy as np
import pandas as pd

In [14]:
# Récupération du jeu de données
df = pd.read_csv('preprocessed_data.csv')
df.tail()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,HOME,WIN,EFF,ODDS
2639,1610612755,Philadelphia76ers,22100009,20211020,0.518,0.520,0.800,20.0,1,1,145,1.59
2640,1610612747,LosAngelesLakers,22100002,20211019,0.474,0.357,0.474,-7.0,0,0,114,1.62
2641,1610612749,MilwaukeeBucks,22100001,20211019,0.457,0.378,0.778,23.0,0,1,155,1.80
2642,1610612744,GoldenStateWarriors,22100002,20211019,0.441,0.359,0.833,7.0,1,1,138,2.40
2643,1610612751,BrooklynNets,22100001,20211019,0.440,0.531,0.565,-23.0,1,0,110,2.05


In [15]:
# Remplacement des valeurs par des moyennes mobiles
def replace_with_rolling_mean(df, window=10):
    col_number = 0;
    columns_to_average = ["FG_PCT", "FG3_PCT", "FT_PCT", "PLUS_MINUS", "EFF", "ODDS"]
    for col in columns_to_average:
        if col_number < window:
            df[col] = df[col].expanding().mean()
        else:
            df[col] = df[col].rolling(window=window).mean()

In [16]:
replace_with_rolling_mean(df, window=5)
df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,HOME,WIN,EFF,ODDS
0,1610612744,GoldenStateWarriors,42100406,20220616,0.41300,0.413000,1.000000,13.000000,1,1,125.00,2.550
1,1610612738,BostonCeltics,42100406,20220616,0.41900,0.403000,0.958500,0.000000,0,0,115.00,2.060
2,1610612738,BostonCeltics,42100405,20220613,0.41700,0.383333,0.864667,-3.333333,1,0,107.00,2.190
3,1610612744,GoldenStateWarriors,42100405,20220613,0.42925,0.343750,0.865250,0.000000,0,1,110.75,2.045
4,1610612738,BostonCeltics,42100404,20220610,0.42340,0.354000,0.839600,-2.000000,0,0,109.40,1.958


In [17]:
# Sélection des caractéristiques et de la cible
X = df[["TEAM_NAME", "FG_PCT", "FG3_PCT", "FT_PCT", "PLUS_MINUS", "HOME", "EFF", "ODDS"]].copy()
y = df["WIN"]

# Encodage des variables catégorielles
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X[X.columns[X.columns.get_loc('TEAM_NAME')]] = labelencoder.fit_transform(X['TEAM_NAME'])
X.head()

,TEAM_NAME,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,HOME,EFF,ODDS
0,9,0.41300,0.413000,1.000000,13.000000,1,125.00,2.550
1,1,0.41900,0.403000,0.958500,0.000000,0,115.00,2.060
2,1,0.41700,0.383333,0.864667,-3.333333,1,107.00,2.190
3,9,0.42925,0.343750,0.865250,0.000000,0,110.75,2.045
4,1,0.42340,0.354000,0.839600,-2.000000,0,109.40,1.958


In [18]:
# Split des données en train et test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Normalisation des données
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train[:5]

array([[ 7.74030375e-01,  6.85543626e-01,  4.50071549e-01,
        -2.01031731e-01,  3.66935851e-02, -9.91056540e-01,
         7.08556772e-01,  4.07591618e-01],
       [-3.06546683e-02, -3.51456015e+00, -9.25730704e-01,
         1.43344631e+00, -2.23983636e+00,  1.00902417e+00,
        -3.07933727e+00, -3.04238003e+00],
       [-4.90474693e-01,  8.57748585e-01,  1.39770784e+00,
         4.62761665e-01, -2.36753864e-03,  1.00902417e+00,
         5.43893070e-01,  5.90301195e-01],
       [-1.64002476e+00,  7.19414700e-01,  9.12116840e-01,
         3.92896828e-01,  2.43322964e-02,  1.00902417e+00,
         6.60604933e-01,  8.38262034e-01],
       [ 1.99255344e-01,  8.23571366e-01,  9.11767626e-01,
         9.27855233e-02, -1.60965206e-02, -9.91056540e-01,
         7.17509285e-01,  6.22736835e-01]])

In [19]:
# Création du modèle et entraînement
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

# Prédiction des résultats
y_pred = clf.predict(X_test)

# Matrice de confusion et score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Matrice de confusion : [[', cm[0][0], ',', cm[0][1], '] [', cm[1][0], ',', cm[1][1], ']]')
print('Score :', clf.score(X_test, y_test))

Matrice de confusion : [[ 138 , 133 ] [ 82 , 176 ]]
Score : 0.5935727788279773
